# 作業 : (Kaggle)鐵達尼生存預測

# [作業目標]
- 試著完成三種不同特徵類型的三種資料操作, 觀察結果
- 思考一下, 這三種特徵類型, 哪一種應該最複雜/最難處理

# [作業重點]
- 完成剩餘的八種 類型 x 操作組合 (In[6]~In[13], Out[6]~Out[13])
- 思考何種特徵類型, 應該最複雜

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
# 載入基本套件
import pandas as pd
import numpy as np

# 讀取訓練與測試資料
data_path = '/content/drive/My Drive/機器學習百日馬拉松/day7/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')
df_train.shape

(891, 12)

In [11]:
# 把資料撈出來
df_train
# 總共有 PassengerId,	Survived,	Pclass,	Name,	Sex,	Age,	SibSp,	Parch,	Ticket,	Fare,	Cabin,	Embarked，共12欄
# PassengerId沒有意義，所以可以drop

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [12]:
df_test
# 這組沒有Survived這一欄

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [13]:
# 重組資料成為訓練 / 預測用格式
# drop丟掉東西只後就沒有了，所以只可以丟一次
train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()
# df--->1309 rows × 10 columns

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [14]:
# 秀出資料欄位的類型與數量
dtype_df = df.dtypes.reset_index()
dtype_df.columns = ["Count", "Column Type"]
dtype_df = dtype_df.groupby("Column Type").aggregate('count').reset_index()
dtype_df

,Column Type,Count
0,int64,3
1,float64,2
2,object,5


In [17]:
#確定只有 int64, float64, object 三種類型後, 分別將欄位名稱存於三個 list 中
int_features, float_features, object_features = [], [], []
for dtype, feature in zip(df.dtypes, df.columns):
  #  print(dtype,feature)
  if dtype == 'int64':
    int_features.append(feature)
  elif dtype == 'object':
    object_features.append(feature)
  else:
    float_features.append(feature)
print(f'{len(int_features)} integer feature: ', int_features)
print(f'{len(float_features)} float feature: ', float_features)
print(f'{len(object_features)} object feature: ', object_features)

3 integer feature:  ['Pclass', 'SibSp', 'Parch']
2 float feature:  ['Age', 'Fare']
5 object feature:  ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']


# 作業1 
* 試著執行作業程式，觀察三種類型 (int / float / object) 的欄位分別進行( 平均 mean / 最大值 Max / 相異值 nunique )  
中的九次操作會有那些問題? 並試著解釋那些發生Error的程式區塊的原因?  

# 作業2
* 思考一下，試著舉出今天五種類型以外的一種或多種資料類型，你舉出的新類型是否可以歸在三大類中的某些大類?  
所以三大類特徵中，哪一大類處理起來應該最複雜?

In [72]:
# 作業1:請依序列出 三種特徵類型 (int / float / object) x 三種方法 (平均 mean / 最大值 Max / 相異值 nunique) 的其餘操作
# 通常int和float可以做出三種方法,object無法做出mean，原因是因為他不是數字類型

print('int_features mean:\n',df[int_features].mean(),'\n')
print('int_features max:\n',df[int_features].max(),'\n')
print('int_feature nunique:\n',df[int_features].nunique(),'\n')

int_features mean:
 Pclass    2.294882
SibSp     0.498854
Parch     0.385027
dtype: float64 

int_features max:
 Pclass    3
SibSp     8
Parch     9
dtype: int64 

int_feature nunique:
 Pclass    3
SibSp     7
Parch     8
dtype: int64 



In [73]:
print('float_features mean:\n',df[float_features].mean(),'\n')
print('float_features max:\n',df[float_features].max(),'\n')
print('float_features nunique:\n',df[float_features].nunique(),'\n')

float_features mean:
 Age     29.881138
Fare    33.295479
dtype: float64 

float_features max:
 Age      80.0000
Fare    512.3292
dtype: float64 

float_features nunique:
 Age      98
Fare    281
dtype: int64 



In [74]:
print('object_features mean:\n',df[object_features].mean(),'\n')
print('object_features max:\n',df[object_features].max(),'\n')
print('object_features nunique:\n',df[object_features].nunique(),'\n')

object_features mean:
 Series([], dtype: float64) 

object_features max:
 Name      van Melkebeke, Mr. Philemon
Sex                              male
Ticket                      WE/P 5735
dtype: object 

object_features nunique:
 Name        1307
Sex            2
Ticket       929
Cabin        186
Embarked       3
dtype: int64 



In [58]:
# 作業2:思考一下，試著舉出今天五種類型以外的一種或多種資料類型，你舉出的新類型是否可以歸在三大類中的某些大類?--->
# list屬於容器類型--->裡面的資料有可能是int64或<U1---->應該屬於數值型特徵
# Dict屬於容器類型--->裡面的資料是object---->應該屬於類別型特徵

# 所以三大類特徵中，哪一大類處理起來應該最複雜?--->
# 時間型特徵

a = {'a':'123', 'b':'456'}
b = np.array(a)
print(type(b))
print(b.dtype,'\n')

a2 = [1, 2, 3, 4]
b2 = np.array(a2)
print(type(b2))
print(b2.dtype,'\n')

a3 = ['asd', 'erttt', 'hsudfi']
b3 = np.array(a3)
print(type(b3))
print(b3.dtype,'\n')

print(np.typeDict['a'])
print(np.typeDict['double'])
print(np.typeDict['I'])

<class 'numpy.ndarray'>
object 

<class 'numpy.ndarray'>
int64 

<class 'numpy.ndarray'>
<U6 

<class 'numpy.bytes_'>
<class 'numpy.float64'>
<class 'numpy.uint32'>
